In [13]:
from ase import Atoms
from ase.build import bulk

In [39]:
from ase import Atoms
from ase.build import bulk
import numpy as np
from pyevtk.hl import pointsToVTK
from pyevtk.hl import gridToVTK#, pointsToVTKAsTIN
from aptstructure.funct_syn_data import  *
from ase.visualize import view
import h5py


In [15]:
def is_inside_halftorus(position, center, major_radius, minor_radius):
    displacement = position - center
    r_cylindrical = np.linalg.norm(displacement[:2])
    is_in_torus = (r_cylindrical - major_radius)**2 + displacement[2]**2 < minor_radius**2
    return is_in_torus and displacement[0] > 0  # for half torus


def is_inside_sphere(position, center, radius):
    displacement = position - center
    return np.linalg.norm(displacement) < radius



In [16]:
supercell=APT_structure(Type="l12", size=150, ratio_SOL=0.0,  m=0, sigma_x_y=0., sigma_z=0.,eff=1,angles=[0,0,0],rot=False  ) #150

/Users/alaukiksaxena/tensorflow-test/Development_20230613/Dislocations_20230613/env_dis/lib/python3.8/site-packages/ase/atoms.py:967: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  warnings.warn('Use get_global_number_of_atoms() instead',


In [17]:
%%time
center_torus = supercell.cell.diagonal() / 2
major_radius = 100  # Adjust based on your requirements
minor_radius = 30   # Adjust based on your requirements

center_sphere1 = center_torus - [0,major_radius, 0]  # Adjust based on your torus orientation
center_sphere2 = center_torus + [0,major_radius,  0]  # Adjust based on your torus orientation
atom_id = []
for atom in supercell:
    if not(is_inside_halftorus(atom.position, center_torus, major_radius, minor_radius) or 
            is_inside_sphere(atom.position, center_sphere1, minor_radius) or
            is_inside_sphere(atom.position, center_sphere2, minor_radius)):
        atom.symbol = 'Al'
        atom_id.append(atom.index)


CPU times: user 1min 37s, sys: 1.11 s, total: 1min 38s
Wall time: 1min 38s


In [23]:
atom_id = np.array(atom_id)

In [24]:
int(len(atom_id)*0.1)

1343972

In [25]:
cu_idrand = atom_id[np.random.randint(0,len(atom_id),int(len(atom_id)*0.1))]

In [26]:
cu_idrand

array([ 1327226,   648279, 13190457, ...,  6778693,  7031830,  2877248])

In [27]:
supercell.symbols[cu_idrand] = "Cu"

In [33]:
element_typ = supercell.get_atomic_numbers()

In [34]:
pos_cell = supercell.get_positions()

In [35]:
element_typ_mass = np.where(element_typ==13, 26.98, 62.0)

In [36]:
element_typ_mass

array([62.  , 26.98, 26.98, ..., 62.  , 26.98, 26.98])

In [37]:
data_pos_mass = np.hstack((pos_cell, element_typ_mass.reshape(-1,1),  element_typ.reshape(-1,1)))

In [40]:
with h5py.File("iso_Al_Cu_tourus_150_random_diffuse_solidsol_diff.h5", "w") as hdfr:
    
    hdfr.create_dataset("0", data = data_pos_mass)

In [12]:
center_torus

array([303.75, 303.75, 303.75])

In [13]:
supercell.get_positions()

array([[  0.   ,   0.   ,   0.   ],
       [  0.   ,   2.025,   2.025],
       [  2.025,   0.   ,   2.025],
       ...,
       [603.45 , 605.475, 605.475],
       [605.475, 603.45 , 605.475],
       [605.475, 605.475, 603.45 ]])

In [41]:
from pyevtk.hl import pointsToVTK
from pyevtk.hl import gridToVTK#, pointsToVTKAsTIN
image = supercell.get_positions()
FILE_PATH1 = "20230615_dis_half_tourus_apt_noise"
print(FILE_PATH1)
x = np.ascontiguousarray(image[:,0]/10)
y= np.ascontiguousarray(image[:,1]/10)
z = np.ascontiguousarray(image[:,2]/10)
label = np.ascontiguousarray( supercell.get_atomic_numbers())
pointsToVTK(FILE_PATH1,x,y,z, data = {"label" : label }  )

20230615_dis_half_tourus_apt_noise


'/Users/alaukiksaxena/tensorflow-test/Development_20230613/Dislocations_20230613/Synthetic_Data_Point_clouds/20230615_dis_half_tourus_apt_noise.vtu'

In [ ]:
# making a cylinder
"""import numpy as np

def is_inside_cylinder(position, center, axis, radius):
    displacement = position - center
    distance_to_axis = np.linalg.norm(displacement - np.dot(displacement, axis) * axis)
    return distance_to_axis < radius

center = supercell.cell.diagonal() / 2
axis = np.array([0, 0, 1])  # Change this to the desired axis of the cylinder
radius = 10  # Change this to the desired radius

for atom in supercell:
    if is_inside_cylinder(atom.position, center, axis, radius):
        atom.symbol = 'Cu'""";

#Making a torus

"""
def is_inside_torus(position, center, major_radius, minor_radius):
    displacement = position - center
    # Keep only x and y for cylindrical coordinates
    r_cylindrical = np.linalg.norm(displacement[:2])
    return (r_cylindrical - major_radius)**2 + displacement[2]**2 < minor_radius**2

center = supercell.cell.diagonal() / 2
major_radius = 100  # Adjust based on your requirements
minor_radius = 30 # Adjust based on your requirements

for atom in supercell:
    if is_inside_torus(atom.position, center, major_radius, minor_radius):
        atom.symbol = 'Cu'
        
"""